In [5]:
import pandas as pd 
import numpy as np
import os
import pyprind
import matplotlib.pyplot as plt

# To change date to datetime
from datetime import datetime
import re 

from collections import Counter
import string
import scipy.sparse

# Gensim libraries
from gensim import corpora
from gensim.models.ldamulticore import LdaMulticore
from gensim.models import CoherenceModel
from gensim import matutils

ModuleNotFoundError: No module named 'pyprind'

In [13]:
import pyLDAvis

C:\Users\ADMIN\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Read data and convert thr reviews to list

In [14]:
dataframe = pd.read_csv('clean_reviews.csv')
dataframe.head()

C:\Users\ADMIN\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,review,sentiment,processed_text
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1,teenager martha moxley maggie grace move high...
1,This is a terrible film. Angie Dickenson is a ...,0,terrible film angie dickenson class act always...
2,"This is supposed to be Charlie's masterpiece, ...",0,supposed charlies masterpiece contend actually...
3,"In order for a thriller to elicit fear, suspen...",0,order thriller elicit fear suspense emotion st...
4,The Perfect Son is a story about two 30-someth...,1,perfect son story two something brother one se...


In [15]:
# Extract lematized words length not less than 2
def minimize_review(text): # text
    rev_text = ([token for token in text.split(' ') if len(token) > 3])
    return rev_text

C:\Users\ADMIN\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [16]:
clean_reviews = []
for review in dataframe.processed_text:
  clean_reviews.append(minimize_review(review))
len(clean_reviews)

C:\Users\ADMIN\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


49999

In [19]:
%%time
# Create Dictionary
id2word_1 = corpora.Dictionary(clean_reviews)
# Create Corpus: Term Document Frequency
corpus_1 = [id2word_1.doc2bow(review) for review in clean_reviews]
 # Build LDA model
ldamodel = LdaMulticore(corpus= corpus_1, num_topics =8, id2word=id2word_1,chunksize=2000, passes=10,per_word_topics=True)

C:\Users\ADMIN\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
Wall time: 6min 42s


In [20]:
from pprint import pprint
pprint(ldamodel.show_topics(formatted=False))

C:\Users\ADMIN\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[(0,
  [('film', 0.041057535),
   ('movie', 0.011891212),
   ('story', 0.007002972),
   ('great', 0.00594792),
   ('like', 0.0058649695),
   ('character', 0.0058461796),
   ('time', 0.005667807),
   ('well', 0.005188769),
   ('good', 0.0048797065),
   ('performance', 0.004306048)]),
 (1,
  [('show', 0.012091825),
   ('film', 0.0072194836),
   ('life', 0.006892898),
   ('episode', 0.0054873885),
   ('story', 0.0054774564),
   ('series', 0.0050922595),
   ('character', 0.0049469364),
   ('love', 0.0048466558),
   ('time', 0.004604958),
   ('year', 0.0045240372)]),
 (2,
  [('film', 0.006939157),
   ('world', 0.0060838605),
   ('people', 

In [22]:
# Compute Perplexity
#It's a measure of how good the model is. The lower the better. Perplexity is a negative value
print('\nPerplexity: ', ldamodel.log_perplexity(corpus_1))  

# Compute Coherence Score
# The coherence score is used in assessing the quality of the learned topics, the closer to 1 the better
coherence_model_lda = CoherenceModel(model=ldamodel, texts=clean_reviews, dictionary=id2word_1, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nBasic Ldamodel Coherence Score: ', coherence_lda)

C:\Users\ADMIN\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)

Perplexity:  -8.72404326244176

Basic Ldamodel Coherence Score:  0.2981596615355011


## Choosing the optimal number of topics to optimize coherence

In [ ]:
num_topics = []
coherence = []
for i in range(4,15,2):
    ldamodel = LdaMulticore(corpus= corpus_1, num_topics =i, id2word=id2word_1,chunksize=2000, passes=30,per_word_topics=True)
    coherence_model_lda = CoherenceModel(model=ldamodel, texts=clean_reviews, dictionary=id2word_1, coherence='c_v')
    num_topics.append(i)
    coherence.append(coherence_model_lda)


In [ ]:
plt.plot(num_topics, coherence)
plt.title("Coherence value score with the number of topics")
plt.xlabel("Number of Topics")
plt.ylabel("Coherence score")
plt.show()